<a href="https://colab.research.google.com/github/takeru0208/sample_deep_learning/blob/feature/sample_neuralnet1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About this program
this program refers to the site [here](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) #6*6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # view は自動サイズ調整
        # １つ目の引数に-1を指定することで２つ目の引数で指定した値にサイズ数を自動で調整してくれる。
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
params = list(net.parameters())
#print model's parameters with its name and size 

for name, param in net.named_parameters():
    if param.requires_grad:
      print(name)
      print(param.size())

conv1.weight
torch.Size([6, 1, 3, 3])
conv1.bias
torch.Size([6])
conv2.weight
torch.Size([16, 6, 3, 3])
conv2.bias
torch.Size([16])
fc1.weight
torch.Size([120, 576])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])


In [12]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0648, -0.1313, -0.0470,  0.1288,  0.0300,  0.1674,  0.0472,  0.1848,
          0.1111, -0.0317]], grad_fn=<AddmmBackward>)


In [13]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [15]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
# viewの-1はよくわからないけど自動で調整してほしいところに入れる。
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

print(target.size())
loss = criterion(output, target)
print(loss)

torch.Size([1, 10])
tensor(1.6136, grad_fn=<MseLossBackward>)


input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d\
     -> view -> linear -> relu -> linear -> relu -> linear\
      -> MSELoss\
      -> loss

In [27]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])# Linear

In [28]:
net.zero_grad()

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward")
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0138, -0.0033,  0.0269, -0.0087, -0.0049,  0.0059])


In [29]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()